# MiniGPT

In [5]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [1]:
import keras 
import pathlib

extract_dir = keras.utils.get_file(
    fname="mini_c4.zip",
    origin=(
        "https://hf.co/datasets/mattdangerw/mini-c4/resolve/main/mini-c4.zip"
    ),
    extract=True,
)
extract_dir = pathlib.Path(extract_dir).parent / "mini-c4"

1505728294/1505728294 ━━━━━━━━━━━━━━━━━━━━ 26s 0us/step


In [ ]:
with open(extract_dir / "shard0.txt", "r") as f:
    print(f.readline().replace("\\n", "\n")[:100])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yuhaochen/.keras/datasets/mini-c4/shard0.txt'